## Simple Linear Regression - OLS 

Lets assume that the linear regression equation we are trying to derive is
<br>
Price = 2000000 + $Square feet*3000$
<br>
where $\beta_0$ = 2000000
<br>
and $\beta_1$ = 3000
<br>
<br>
We will go ahead and create values for x and y and then try to predict them using Linear regression formulae

In [101]:
import numpy as np
import math
import pandas as pd

np.random.seed(0)
x = np.random.choice(range(600,2500),5000)
y = [2000000 + each*3000 for each in x]

data = pd.DataFrame({'square_feet':x,'price':y})


In [102]:
data.head()

,square_feet,price
0,1284,5852000
1,1159,5477000
2,2253,8759000
3,1816,7448000
4,1435,6305000


Solving the Linear Regression using OLS method using formulas for $\beta_0$ and $\beta_1$
<br>
<br>
$\beta_1$ = $r * \sigma_Y / \sigma_x$
<br>
and $\beta_0$ = $\bar{Y} - \beta_1 * \bar{X}$
<br>
where r is the pearson correlation coefficient
<br>
and $\sigma_Y$ is standard deviation of Y and $\sigma_x$ is standard deviation of x
<br>
and $\bar{X}$ and $\bar{Y}$ are mean of X and Y 

In [103]:
import sys

sys.path.append('../')

from CommonFuncs.CommonFunctions import DescriptiveAnalysis
da = DescriptiveAnalysis(data)
r = da.pearson_corr_cont('square_feet','price')

sigma_x = data['square_feet'].std()
sigma_y = data['price'].std()
beta1 = r * sigma_y/sigma_x
Y_bar = data['price'].mean()
X_bar = data['square_feet'].mean()
beta0= Y_bar - beta1 * X_bar
print("Beta 0 is {}".format(str(beta0)))
print("Beta 1 is {}".format(str(beta1)))

1.0
Beta 0 is 2000000.0000000028
Beta 1 is 2999.999999999998


We saw how we derived beta 0 and beta 1 using formulae. Now we will see the same using standard python packages like statsmodels.

## Simple Linear Regression using Statsmodels

In [104]:
import statsmodels.api as sm
X = data['square_feet']
Y = data['price']
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.params

C:\Users\Asha\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


const          2000000.0
square_feet       3000.0
dtype: float64

## Linear Regression using Gradient Descent

In gradient descent we initialize beta0 and beta1 with random values. We calculate the error between the observed and expected at every step. We then adjust the beta0 and beta1 values to minimize the error (or we can say we backpropogate the gradient of the error)
<br>
Lets initialize beta0 = 10 and beta1 = 10.
<br>
We now calculate the Y based on this equation
<br>
Y = beta0 + beta1 * x
<br>
<br>
if x = 684
<br>
We find $\hat{Y} = 6850$
<br>
Actual Y is 4052000
<br>
<br>
SSE = $1/n * (Y - \hat{Y})^2$
<br>
So SSE is 8181619261250
<br>
To get the adjustment for $\beta_0$ and $\beta_1$, we take the gradient of the error by $\beta_0$ and $\beta_1$ and adjust it by that term.
<br>
<br>
Adjustment for $\beta_0$ :
<br>
<br>
$\frac{d(SSE)}{d\beta_0} = \frac{d(1/2*(Y - \hat{Y})^2)}{d\beta_0} = \frac{d(1/2*(Y - (\beta_0 + \beta_1 * x))^2)}{d\beta_0} = -2/n\sum{(Y_i-\hat{Y_i}})$
<br>
<br>
Adjustment for $\beta_1$ :
<br>
<br>
$\frac{d(SSE)}{d\beta_1} = \frac{d(1/2*(Y - \hat{Y})^2)}{d\beta_1} = \frac{d(1/2*(Y - (\beta_0 + \beta_1 * x))^2)}{d\beta_1} = -2/n\sum{(Y_i-\hat{Y_i})}*x_i$
<br>
<br>
$\beta_0 (step1) = \beta_0 - error * alpha$
<br>
where alpha is the learning rate (or how fast should the error be adjusted )
<br>
<br>
if alpha = 0.1
<br>
then $\beta_0 (step1) = 10 - (4052000-6850)* 0.1 = 404525$
<br>
$\beta_1(step1) = \beta_1 - error * alpha * x$
<br>
Here we can see $\beta_1$ is adjusted by a value proportional to x
<br>
so $\beta_1(step1) = 10 - (4052000-6850)* 0.1 * 684 = 276688270$
<br>
<br>
We use this process for the batch of records over and over again, till we attain convergence.
Here we use an alpha of 0.01. Depending on the speed of convergence and if we see the errors wildly fluctuating, then we need to tune this value of alpha.
Do note, that we need to scale the values before doing gradient descent as it aids in faster convergence. Here we have done min max scaling. For each value of x we calculate (x_i - min_x ) / (max_x - min_x).
We do the same for Y.

In [181]:


def get_error(y,x,beta0,beta1):
    y_hat = beta0 + beta1 * x
    error = (y - y_hat)
    return error

def get_sse(y,x,beta0,beta1):
    y_hat = beta0 + beta1 * x
    sse = 1/2*(y - y_hat)**2
    return sse

beta0 = 0
beta1 = 0

x = list(data['square_feet'])
y = list(data['price'])
alpha = 0.01

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
data_scaled = scaler.fit_transform(data)
x = np.array(list(data_scaled[:,1]))
y = np.array(list(data_scaled[:,0]))
n = len(x)

iterations = 5000
for i in range(iterations):
    print("Iteration {}".format(str(i)))
    print("beta0 : ",str(beta0))
    print("beta1 : ",str(beta1))
    y_pred = beta1*x + beta0    
    beta1_update = (-2/n) * sum(x * (y - y_pred))
    beta0_update = (-2/n) * sum(y - y_pred)
    beta0 = beta0 - alpha*beta0_update
    beta1 = beta1 - alpha*beta1_update
    error = get_error(y_val,x_val,beta0,beta1)
    sse = get_sse(y_val,x_val,beta0,beta1)
    print("Y-Y Predicted) : ",str(error))
    print("SSE :",str(sse))



Iteration 0
beta0 :  0
beta1 :  0
Y-Y Predicted) :  0.4984341406370181
SSE : 0.12421829627628135
Iteration 1
beta0 :  0.010000358083201709
beta1 :  0.006669636759570533
Y-Y Predicted) :  0.48536060354873756
SSE : 0.11778745773859739
Iteration 2
beta0 :  0.019734010248858777
beta1 :  0.013194782302844169
Y-Y Predicted) :  0.47261909243617733
SSE : 0.11168440326759797
Iteration 3
beta0 :  0.02920773557922494
beta1 :  0.01957906748842695
Y-Y Predicted) :  0.4602011887547023
SSE : 0.10589256706562056
Iteration 4
beta0 :  0.038428141265022676
beta1 :  0.02582603116516683
Y-Y Predicted) :  0.4480986873943272
SSE : 0.10039621682225948
Iteration 5
beta0 :  0.04740166696340431
beta1 :  0.03193912250480226
Y-Y Predicted) :  0.4363035912685499
SSE : 0.09518041187691692
Iteration 6
beta0 :  0.05613458904542665
beta1 :  0.03792170327547162
Y-Y Predicted) :  0.4248081060403733
SSE : 0.09023096347880451
Iteration 7
beta0 :  0.06463302473584019
beta1 :  0.043777050057582874
Y-Y Predicted) :  0.4136046

Y-Y Predicted) :  0.0734963428760444
SSE : 0.002700856208076541
Iteration 69
beta0 :  0.3091643505615101
beta1 :  0.2523943859692222
Y-Y Predicted) :  0.07123360747956092
SSE : 0.002537113417276079
Iteration 70
beta0 :  0.3104573873955995
beta1 :  0.2542888896420476
Y-Y Predicted) :  0.06902876469663499
SSE : 0.0023824851777717007
Iteration 71
beta0 :  0.31170561777788897
beta1 :  0.2561578268321799
Y-Y Predicted) :  0.0668803460950107
SSE : 0.0022364903468942068
Iteration 72
beta0 :  0.3129101935113963
beta1 :  0.2580018161393344
Y-Y Predicted) :  0.0647869204722088
SSE : 0.0020986725321361535
Iteration 73
beta0 :  0.3140722371768604
beta1 :  0.2598214605189481
Y-Y Predicted) :  0.0627470929116467
SSE : 0.001968598834431412
Iteration 74
beta0 :  0.3151928428736349
beta1 :  0.26161734767875533
Y-Y Predicted) :  0.06075950386268825
SSE : 0.0018458586548200139
Iteration 75
beta0 :  0.31627307694179885
beta1 :  0.2633900504653076
Y-Y Predicted) :  0.0588228282440168
SSE : 0.00173006256131

Y-Y Predicted) :  -0.0034041422061857896
SSE : 5.794092079967727e-06
Iteration 149
beta0 :  0.332823090068777
beta1 :  0.3564129814906781
Y-Y Predicted) :  -0.0036772753045674067
SSE : 6.7611768327906565e-06
Iteration 150
beta0 :  0.3326027289101949
beta1 :  0.35737712304826486
Y-Y Predicted) :  -0.003943010393188673
SSE : 7.773665480396946e-06
Iteration 151
beta0 :  0.33237713321396556
beta1 :  0.3583370378223711
Y-Y Predicted) :  -0.004201535655577571
SSE : 8.826450932544824e-06
Iteration 152
beta0 :  0.33214644994019044
beta1 :  0.35929280635135824
Y-Y Predicted) :  -0.004453034503462572
SSE : 9.914758144514077e-06
Iteration 153
beta0 :  0.3319108223043563
beta1 :  0.36024450716677237
Y-Y Predicted) :  -0.004697685697751974
SSE : 1.1034125457431727e-05
Iteration 154
beta0 :  0.33167038987229663
beta1 :  0.3611922168441765
Y-Y Predicted) :  -0.004935663466446583
SSE : 1.2180386927007748e-05
Iteration 155
beta0 :  0.3314252886527452
beta1 :  0.3621360100526943
Y-Y Predicted) :  -0.005

Y-Y Predicted) :  -0.011863733324163461
SSE : 7.03740841934333e-05
Iteration 224
beta0 :  0.30839569101068365
beta1 :  0.4206644160013355
Y-Y Predicted) :  -0.011888568486694617
SSE : 7.066903033141418e-05
Iteration 225
beta0 :  0.3080213404807974
beta1 :  0.421444306567077
Y-Y Predicted) :  -0.011912345226922527
SSE : 7.095198440269196e-05
Iteration 226
beta0 :  0.3076466777765857
beta1 :  0.4222227391853803
Y-Y Predicted) :  -0.011935090943449467
SSE : 7.122319791420474e-05
Iteration 227
beta0 :  0.3072717237215316
beta1 :  0.4229997267020802
Y-Y Predicted) :  -0.011956832339502732
SSE : 7.148291979748918e-05
Iteration 228
beta0 :  0.3068964985941854
beta1 :  0.4237752816690923
Y-Y Predicted) :  -0.011977595440566091
SSE : 7.17313962689348e-05
Iteration 229
beta0 :  0.3065210221420028
beta1 :  0.42454941635182264
Y-Y Predicted) :  -0.011997405611562262
SSE : 7.196887070417282e-05
Iteration 230
beta0 :  0.3061453135948319
beta1 :  0.4253221427363898
Y-Y Predicted) :  -0.01201628757360

Y-Y Predicted) :  -0.012046320249558096
SSE : 7.255691577745672e-05
Iteration 297
beta0 :  0.2811097867559847
beta1 :  0.47432970895503684
Y-Y Predicted) :  -0.012036663941897174
SSE : 7.244063942508381e-05
Iteration 298
beta0 :  0.28074448216501546
beta1 :  0.4750245403233522
Y-Y Predicted) :  -0.012026862991145082
SSE : 7.232271670388762e-05
Iteration 299
beta0 :  0.28037953510337504
beta1 :  0.47571839059555077
Y-Y Predicted) :  -0.012016921577261974
SSE : 7.220320209703221e-05
Iteration 300
beta0 :  0.2800149482317893
beta1 :  0.4764112627397661
Y-Y Predicted) :  -0.012006843773558873
SSE : 7.208214870132474e-05
Iteration 301
beta0 :  0.2796507241280872
beta1 :  0.477103159677727
Y-Y Predicted) :  -0.011996633549402613
SSE : 7.195960825932617e-05
Iteration 302
beta0 :  0.27928686528932295
beta1 :  0.4777940842858962
Y-Y Predicted) :  -0.011986294772851513
SSE : 7.183563119084375e-05
Iteration 303
beta0 :  0.27892337413384377
beta1 :  0.47848403939658
Y-Y Predicted) :  -0.0119758312

...

Y-Y Predicted) :  -3.564792187338295e-05
SSE : 6.353871669454072e-10
Iteration 4732
beta0 :  0.0008149998563130044
beta1 :  0.9984773772391966
Y-Y Predicted) :  -3.560097926913475e-05
SSE : 6.33714862460681e-10
Iteration 4733
beta0 :  0.0008139266320204091
beta1 :  0.9984793822895321
Y-Y Predicted) :  -3.555409848077229e-05
SSE : 6.320469593902272e-10
Iteration 4734
beta0 :  0.0008128548209923361
beta1 :  0.9984813846995375
Y-Y Predicted) :  -3.550727942691623e-05
SSE : 6.303834461505541e-10
Iteration 4735
beta0 :  0.000811784421367741
beta1 :  0.9984833844726896
Y-Y Predicted) :  -3.546052202618721e-05
SSE : 6.287243111848543e-10
Iteration 4736
beta0 :  0.0008107154312880321
beta1 :  0.9984853816124606
Y-Y Predicted) :  -3.5413826197427944e-05
SSE : 6.270695429708169e-10
Iteration 4737
beta0 :  0.000809647848897065
beta1 :  0.9984873761223184
Y-Y Predicted) :  -3.536719185970316e-05
SSE : 6.254191300205268e-10
Iteration 4738
beta0 :  0.000808581672341139
beta1 :  0.9984893680057262
Y-

Y-Y Predicted) :  -3.2592647706164435e-05
SSE : 5.311403422490729e-10
Iteration 4800
beta0 :  0.0007451487155928633
beta1 :  0.9986078765710733
Y-Y Predicted) :  -3.254972840871506e-05
SSE : 5.297424097405562e-10
Iteration 4801
beta0 :  0.0007441674740662862
beta1 :  0.9986097097746875
Y-Y Predicted) :  -3.2506865629056136e-05
SSE : 5.283481565127557e-10
Iteration 4802
beta0 :  0.0007431875246776587
beta1 :  0.9986115405642663
Y-Y Predicted) :  -3.2464059292802716e-05
SSE : 5.269575728833052e-10
Iteration 4803
beta0 :  0.0007422088657254416
beta1 :  0.9986133689429882
Y-Y Predicted) :  -3.242130932568088e-05
SSE : 5.255706491957411e-10
Iteration 4804
beta0 :  0.0007412314955103373
beta1 :  0.9986151949140283
Y-Y Predicted) :  -3.237861565341671e-05
SSE : 5.241873758158408e-10
Iteration 4805
beta0 :  0.0007402554123352849
beta1 :  0.9986170184805568
Y-Y Predicted) :  -3.2335978201958326e-05
SSE : 5.22807743138762e-10
Iteration 4806
beta0 :  0.0007392806145054598
beta1 :  0.9986188396457

SSE : 4.510736074827129e-10
Iteration 4862
beta0 :  0.0006866920526929663
beta1 :  0.9987170881798392
Y-Y Predicted) :  -2.9996213303240538e-05
SSE : 4.498864062667523e-10
Iteration 4863
beta0 :  0.0006857877892298852
beta1 :  0.9987187775692544
Y-Y Predicted) :  -2.9956713094025922e-05
SSE : 4.4870232969889204e-10
Iteration 4864
beta0 :  0.0006848847165369713
beta1 :  0.9987204647340143
Y-Y Predicted) :  -2.9917264900314322e-05
SSE : 4.475213695577897e-10
Iteration 4865
beta0 :  0.0006839828330461718
beta1 :  0.9987221496770484
Y-Y Predicted) :  -2.9877868653604978e-05
SSE : 4.4634351764103547e-10
Iteration 4866
beta0 :  0.0006830821371914975
beta1 :  0.9987238324012824
Y-Y Predicted) :  -2.9838524285397128e-05
SSE : 4.451687657651171e-10
Iteration 4867
beta0 :  0.0006821826274090219
beta1 :  0.998725512909638
Y-Y Predicted) :  -2.9799231727301034e-05
SSE : 4.4399710576869227e-10
Iteration 4868
beta0 :  0.0006812843021368776
beta1 :  0.9987271912050332
Y-Y Predicted) :  -2.97599909112

Y-Y Predicted) :  -2.767947001736104e-05
SSE : 3.830765302209944e-10
Iteration 4924
beta0 :  0.0006328212950839787
beta1 :  0.9988177321751013
Y-Y Predicted) :  -2.7643020588108413e-05
SSE : 3.820682936172928e-10
Iteration 4925
beta0 :  0.0006319879707815322
beta1 :  0.9988192890324924
Y-Y Predicted) :  -2.7606619156905765e-05
SSE : 3.8106271063721817e-10
Iteration 4926
beta0 :  0.000631155743833739
beta1 :  0.9988208438397517
Y-Y Predicted) :  -2.7570265660470383e-05
SSE : 3.800597742944562e-10
Iteration 4927
beta0 :  0.0006303246127955586
beta1 :  0.9988223965995787
Y-Y Predicted) :  -2.7533960035852623e-05
SSE : 3.7905947762796467e-10
Iteration 4928
beta0 :  0.0006294945762238544
beta1 :  0.9988239473146697
Y-Y Predicted) :  -2.7497702219880793e-05
SSE : 3.7806181368661855e-10
Iteration 4929
beta0 :  0.0006286656326773893
beta1 :  0.9988254959877173
Y-Y Predicted) :  -2.746149214971627e-05
SSE : 3.770667755444642e-10
Iteration 4930
beta0 :  0.0006278377807168239
beta1 :  0.998827042

Y-Y Predicted) :  -2.554165929857799e-05
SSE : 3.2618817986231775e-10
Iteration 4985
beta0 :  0.0005839456429537784
beta1 :  0.9989090440689697
Y-Y Predicted) :  -2.550802502354621e-05
SSE : 3.2532967030092984e-10
Iteration 4986
beta0 :  0.0005831766800579958
beta1 :  0.9989104806832197
Y-Y Predicted) :  -2.547443503952973e-05
SSE : 3.2447342029161007e-10
Iteration 4987
beta0 :  0.0005824087297632017
beta1 :  0.9989119154056788
Y-Y Predicted) :  -2.544088928813082e-05
SSE : 3.2361942388546474e-10
Iteration 4988
beta0 :  0.0005816417907359626
beta1 :  0.9989133482388382
Y-Y Predicted) :  -2.540738771117379e-05
SSE : 3.2276767515295246e-10
Iteration 4989
beta0 :  0.0005808758616446021
beta1 :  0.9989147791851858
Y-Y Predicted) :  -2.5373930250482957e-05
SSE : 3.2191816817818705e-10
Iteration 4990
beta0 :  0.0005801109411591946
beta1 :  0.9989162082472062
Y-Y Predicted) :  -2.5340516847882633e-05
SSE : 3.2107089705891175e-10
Iteration 4991
beta0 :  0.0005793470279515683
beta1 :  0.9989176

As we see, the SSE goes on reducing and the $\beta_0$ and $\beta_1$ values converge. Given a scaled value of x, this would yield a min max scaled value of y, which has to be rescaled back to the original number using the min max.

## Implementing Stepwise Multiple Linear Regression using Statsmodels 

In multiple linear regression, we need to understand which variables to consider ie. do variable selection after removing multicollinearity. Multicollinearity has a huge bearing on changing the significance values of the variables and adversely impacts the interpretation of the model.
<br>
<br>
The steps for model building include -
<br>
<br>
1.Data processing for missing values, dummy encoding (for python) and label encoding of the target variable
<br>
2.Check for multi-collinearity between predictor variables
<br>
3.Identify the set of variables to keep, by selecting one from a set of multi-collinear variables (Note:There can be other approaches for handling multicollinearity)
<br>
4.Identify the variable with maximum correlation with Y and add that first
<br>
5.At each step, identify the variable to be added next based on the highest partial f-score. (Partial f score gives the partial correlation, which is the incremental effect of the adding a variable. This removes the correlation effect of the other Xs on the new variable,but retains their effect on Y.)
<br>
6.Remove the variables which become insignificant. Also study the effect on the beta values after adding each variable.Monitor if there are drastic changes, as that could indicate non-linear correlations.
<br>

## Work in Progress !